In [2]:

import pandas as pd
import requests
from bs4 import BeautifulSoup as BS

Pull in projected stats for all offensive fantasy football players for the 2021 season and calculate their projected PPR score. Due to the format, I had to remove comma formating for some of the metrics. 

In [3]:
df = pd.read_csv('/Users/alokmadan/Documents/GitHub/2021-VOR-Model/data/all_compiled.csv', index_col = 0)
comma_columns = ['REC_YD','RUSH_YD','PASS_YD']
for column in comma_columns:
    df[column] = df[column].str.replace(',','')
    df[column] = df[column].astype(float)


df['PPR'] = (df['RUSH_YD'] + df['REC_YD'])*0.1 + df['PASS_YD']*.04 + (df['RUSH_TD'] + df['REC_TD'])*6 + df['PASS_TD']*4 + df['REC'] + df['FL']*-2 + df['INTS']*-2
df.sort_values(by='PPR', ascending=False).head()

,Player,Team,POS,RUSH_ATT,RUSH_YD,RUSH_TD,REC,REC_YD,REC_TD,FL,PASS_ATT,CMP,PASS_YD,PASS_TD,INTS,PPR
1,Christian McCaffrey,CAR,RB,297.1,1301.6,11.9,96.8,785.0,4.3,2.5,0.0,0.0,0.0,0.0,0.0,397.660
1,Josh Allen,BUF,QB,110.9,519.7,7.6,0.0,0.0,0.0,5.1,605.1,400.0,4565.6,33.7,13.3,378.194
2,Patrick Mahomes II,KC,QB,55.3,267.2,2.9,0.0,0.0,0.0,2.5,630.5,418.9,5027.8,38.5,9.1,376.032
3,Kyler Murray,ARI,QB,125.8,698.8,7.7,0.0,0.0,0.0,3.7,588.5,389.7,4198.2,26.7,12.9,357.608
4,Lamar Jackson,BAL,QB,152.9,920.2,6.7,0.0,0.0,0.0,4.0,462.1,296.3,3377.9,28.6,12.2,349.336


 Once we have the projected PPR score for each player, we can join our data with scrapped Average draft position and rank data from the fantasy pros website using the beautiful soup library. 

In [4]:
ADP_URL = 'https://www.fantasypros.com/nfl/adp/ppr-overall.php'
res = requests.get(ADP_URL)

soup = BS(res.content, 'html.parser')
table = soup.find('table', attrs={'id':'data'})
adp_df = pd.read_html(str(table))[0]
adp_df['Player'] = adp_df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split(' ')[:-2]))
adp_df['Team'] = adp_df['Player Team (Bye)'].apply(lambda x: x.split()[-2])
adp_df['POS'] = adp_df['POS'].str[:2]
adp_df = adp_df.loc[:, ['Player', 'Team', 'POS', 'Rank']]
adp_df.head()

,Player,Team,POS,Rank
0,Christian McCaffrey,CAR,RB,1
1,Dalvin Cook,MIN,RB,2
2,Alvin Kamara,NO,RB,3
3,Derrick Henry,TEN,RB,4
4,Ezekiel Elliott,DAL,RB,5


Merge projected stats with fantasy pros adp data

In [5]:
adp_cutoff = 100
adp_cutoff_df = adp_df.sort_values(by='Rank')[:adp_cutoff]
adp_cutoff_df = adp_cutoff_df.merge(df.loc[:, ['Player', 'Team', 'POS', 'PPR']], on = ['Player', 'Team', 'POS'])
adp_cutoff_df.head()

,Player,Team,POS,Rank,PPR
0,Christian McCaffrey,CAR,RB,1,397.66
1,Dalvin Cook,MIN,RB,2,335.20
2,Alvin Kamara,NO,RB,3,318.33
3,Derrick Henry,TEN,RB,4,295.47
4,Ezekiel Elliott,DAL,RB,5,278.62


Calculate replacement value for each position

In [6]:
replacement_values = {}
for _, row in adp_cutoff_df.iterrows():
    replacement_values[row['POS']] = row['PPR']

replacement_values

{'RB': 118.11,
 'WR': 193.78,
 'TE': 155.32000000000002,
 'QB': 303.65799999999996}

Calculate each players VOR and create smaller data frame only including player, team, position, projected PPR, and VOR 

In [7]:
vor_df = df.loc[:,['Player', 'Team', 'POS', 'PPR']].rename({'POS': 'Position'}, axis = 1)
vor_df['VOR'] = vor_df.apply(lambda row: row['PPR'] - replacement_values[row['Position']], axis =1 )

In [8]:
vor_df.sort_values(by='VOR', ascending = False).head(50)

,Player,Team,Position,PPR,VOR
1,Christian McCaffrey,CAR,RB,397.660,279.550
2,Dalvin Cook,MIN,RB,335.200,217.090
4,Alvin Kamara,NO,RB,318.330,200.220
3,Derrick Henry,TEN,RB,295.470,177.360
12,Austin Ekeler,LAC,RB,285.630,167.520
9,Saquon Barkley,NYG,RB,281.990,163.880
8,Ezekiel Elliott,DAL,RB,278.620,160.510
6,Aaron Jones,GB,RB,276.660,158.550
1,Travis Kelce,KC,TE,311.920,156.600
7,Jonathan Taylor,IND,RB,268.790,150.680


Export to CSV

In [12]:
vor_df.to_csv('VORmodel.csv')